In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/opt/conda/envs/python2/bin/python'

In [2]:
import pyspark
from pyspark.sql import SQLContext, Row
from pyspark import SparkContext
import numpy as np
sc = SparkContext('local[*]')

In [3]:
tf = sc.textFile('rlm_log_5_channel.csv')

In [4]:
rdd = tf.map(lambda x: (x.split(";")[0], x))
rdd2 = rdd.groupByKey().mapValues(list)

In [5]:
rdd2.take(3)

[(u'Hello 6 from the client 5',
  [u'Hello 6 from the client 5;270417294;5',
   u'Hello 6 from the client 5;270686712;1']),
 (u'Hello 14 from the client 7', [u'Hello 14 from the client 7;497480105;7']),
 (u'Hello 13 from the client 4', [u'Hello 13 from the client 4;468675767;4'])]

In [6]:
hopping2 = rdd2.filter(lambda x: len(x[1]) == 2)
hopping3 = rdd2.filter(lambda x: len(x[1]) == 3)

In [7]:
hopping2.take(3)

[(u'Hello 6 from the client 5',
  [u'Hello 6 from the client 5;270417294;5',
   u'Hello 6 from the client 5;270686712;1']),
 (u'Hello 14 from the client 5',
  [u'Hello 14 from the client 5;500956587;5',
   u'Hello 14 from the client 5;501071454;1']),
 (u'Hello 14 from the client 3',
  [u'Hello 14 from the client 3;500101745;3',
   u'Hello 14 from the client 3;500230643;1'])]

In [8]:
hopping3.take(3)

[(u'Hello 6 from the client 7',
  [u'Hello 6 from the client 7;257049258;7',
   u'Hello 6 from the client 7;258180051;9',
   u'Hello 6 from the client 7;258915193;1']),
 (u'Hello 17 from the client 8',
  [u'Hello 17 from the client 8;594142364;8',
   u'Hello 17 from the client 8;594192920;3',
   u'Hello 17 from the client 8;594402592;1']),
 (u'Hello 7 from the client 4',
  [u'Hello 7 from the client 4;286511726;4',
   u'Hello 7 from the client 4;286557567;9',
   u'Hello 7 from the client 4;286662475;1'])]

In [9]:
latency2 = hopping2.map(lambda x: int(x[1][1].split(";")[1]) - int(x[1][0].split(";")[1])).collect()

In [10]:
latency3 = hopping3.map(lambda x: int(x[1][2].split(";")[1]) - int(x[1][0].split(";")[1])).collect()

In [11]:
print 'mean latency 2: %f' % np.mean(latency2)
print 'mean latency 3: %f' % np.mean(latency3)
print 'Standard deviation latency 2: %f' % np.std(latency2)
print 'Standard deviation latency 3: %f' % np.std(latency3)


mean latency 2: 388840.729167
mean latency 3: 743272.550000
Standard deviation latency 2: 329870.947627
Standard deviation latency 3: 495522.906274


In [12]:
import plotly
from plotly.graph_objs import Scatter, Layout, Bar, Figure, Layout

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np

init_notebook_mode(connected=True)

In [13]:
trace1 = Bar(
    x=['2 Hop', '3 Hop',],
    y=[np.mean(latency2), np.mean(latency3)],
    name='Control',
    error_y=dict(
        type='data',
        array=[np.std(latency2), np.std(latency3)],
        visible=True
    )
)

data = [trace1]
layout = Layout(
    barmode='group'
)
fig = Figure(data=data, layout=layout)
iplot(fig, filename='error-bar-bar')